In [5]:
from sqlalchemy import create_engine, Column, Table, String, MetaData

metadata_obj = MetaData()
engine = create_engine('sqlite:///databases/cats.db', future=True)

cat_breeds_table = Table(
    "cat_breeds",
    metadata_obj,
    Column("name", String(16), primary_key=True),
    Column('origin', String(16), nullable=False),
    Column('temperature',String(16), nullable=False),
    Column('size',String(16), nullable=False),
    Column('social_behavior', String(16), nullable=False)
)

metadata_obj.create_all(engine)

In [8]:
from sqlalchemy import insert

breeds_data = [
    {
        "name" : "Savannah",
        "origin" : 'USA',
        'temperature' : 'high energy',
        'size' : 'Large',
        'social_behavior' : 'sociable'
    },
    
    {
        "name" : "Ragdoll",
        "origin" : 'USA',
        'temperature' : 'gentle',
        'size' : 'Large',
        'social_behavior' : 'sociable'
    },
    
    {
        "name" : "Maine Coon",
        "origin" : 'USA',
        'temperature' : 'friendly',
        'size' : 'Very Large',
        'social_behavior' : 'sociable'
    }
    
]

for breed in breeds_data:
    stmt = insert(cat_breeds_table).values(**breed)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [9]:
with engine.connect() as connection:
    result = connection.exec_driver_sql("SELECT * FROM cat_breeds")
    print(result.fetchall())

[('Savannah', 'USA', 'high energy', 'Large', 'sociable'), ('Ragdoll', 'USA', 'gentle', 'Large', 'sociable'), ('Maine Coon', 'USA', 'friendly', 'Very Large', 'sociable')]


In [10]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["cat_breeds"])

In [11]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(sql_database)

In [12]:
response = query_engine.query('which cat is the biggest?')

In [13]:
print(response)

The Maine Coon is the biggest cat breed.


In [14]:
response.metadata

{'4f5c07d4-e912-4b14-a377-df20e9611b23': {},
 'sql_query': 'SELECT name FROM cat_breeds ORDER BY size DESC LIMIT 1;',
 'result': [('Maine Coon',)],
 'col_keys': ['name']}

In [17]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database,
    context_query_kwargs={"cat_breeds" :(
        "The only columns available are: name, origin, temperament, size, social_behavior. Do not use other columns and foreign keys. \n"
        "Do not attempt to run a query if the column is not among available columns. \n"
        "Do not use unexistant columns in a WHERE part of the query\n"
        
        )} # 모델에게 추가 지시
    )

response = query_engine.query('How long does a Ragdoll cat live?')

In [18]:
response.metadata

{'da7e0bf2-cf89-45a0-ad9f-576f68c8ef76': {},
 'sql_query': "SELECT name, life_expectancy\nFROM cat_breeds\nWHERE name = 'Ragdoll';"}